# Self-RAG Interactive Demo

Interactive demonstration of the complete Self-RAG system for legal analysis.

In [ ]:
import sys
sys.path.append('..')

from src.self_rag.inference import load_pipeline_from_config
import json

## 1. Load Complete Self-RAG Pipeline

In [ ]:
# Load pipeline
print("Loading Self-RAG pipeline...")

pipeline = load_pipeline_from_config(
    retrieval_config_path='../configs/retrieval_config.yaml',
    generator_config_path='../configs/generator_config.yaml',
    retriever_index_dir='../data/embeddings',
    generator_weights_path='../models/generator_lora/final',
)

print("✅ Pipeline loaded!")

## 2. Test with Example Questions

In [ ]:
def demo_question(question):
    """Demonstrate Self-RAG with a question."""
    print(f"\n{'='*80}")
    print(f"Question: {question}")
    print(f"{'='*80}\n")
    
    # Get answer
    result = pipeline.answer_question(question)
    
    # Display formatted response
    formatted = pipeline.format_response(
        result,
        include_passages=True,
        include_reflection=True
    )
    
    print(formatted)
    return result

In [ ]:
# Example 1: Basic negligence question
result1 = demo_question("What are the four elements of negligence?")

In [ ]:
# Example 2: Specific doctrine
result2 = demo_question("What is res ipsa loquitur and when does it apply?")

In [ ]:
# Example 3: Defenses
result3 = demo_question("What is the difference between comparative negligence and assumption of risk?")

## 3. Interactive Question & Answer

Enter your own questions below!

In [ ]:
# Interactive mode
your_question = input("Enter your legal question: ")

if your_question:
    result = demo_question(your_question)

## 4. Analyze Reflection Tokens

Examine the self-verification in action.

In [ ]:
def analyze_reflection(result):
    """Analyze reflection tokens from a result."""
    reflection = result['reflection']
    
    print("\nReflection Token Analysis:")
    print("=" * 50)
    
    print(f"\n📍 Retrieve: {reflection.get('retrieve', 'N/A')}")
    print("   → Did the model decide to retrieve evidence?")
    
    print(f"\n🔍 ISREL (Relevance): {reflection.get('isrel', 'N/A')}")
    print("   → Is the retrieved passage relevant?")
    
    print(f"\n✓ ISSUP (Support): {reflection.get('issup', 'N/A')}")
    print("   → Is the answer supported by evidence?")
    print("   → Hallucination detection!")
    
    print(f"\n⭐ ISUSE (Utility): {reflection.get('isuse', 'N/A')}")
    print("   → Overall response quality (1-5)")
    
    print(f"\n📊 Overall Score: {result['score']:.2f}")
    
    # Hallucination check
    support = reflection.get('issup', '')
    if 'No Support' in support:
        print("\n⚠️  WARNING: Potential hallucination detected!")
    elif 'Fully Supported' in support:
        print("\n✅ Response is fully supported by evidence")

# Analyze previous results
analyze_reflection(result1)

## 5. Batch Processing

Process multiple questions at once.

In [ ]:
# Batch questions
questions = [
    "What is causation in negligence?",
    "What damages can be recovered?",
    "What is professional malpractice?",
]

# Process all
results = pipeline.answer_batch(questions)

# Summary
print("\nBatch Processing Summary:")
print("=" * 80)
for i, (q, r) in enumerate(zip(questions, results), 1):
    print(f"\n{i}. {q}")
    print(f"   Score: {r['score']:.2f}")
    print(f"   Support: {r['reflection'].get('issup', 'N/A')}")
    print(f"   Answer: {r['answer'][:80]}...")

## 6. Export Results

In [ ]:
# Save results for analysis
output = {
    'questions': questions,
    'results': [
        {
            'question': r['question'],
            'answer': r['answer'],
            'reflection': r['reflection'],
            'score': r['score']
        }
        for r in results
    ]
}

with open('../results/demo_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("✅ Results saved to ../results/demo_results.json")

## Summary

Demo complete!
- ✅ Tested Self-RAG on legal questions
- ✅ Analyzed reflection tokens
- ✅ Demonstrated hallucination detection
- ✅ Processed batch questions
- ✅ Exported results

## Key Takeaways

1. **Adaptive Retrieval**: Model decides when to retrieve evidence
2. **Self-Verification**: Reflection tokens provide quality assessment
3. **Hallucination Detection**: ISSUP token identifies unsupported claims
4. **Transparency**: See exactly why the model made each decision

## Next Steps

- Evaluate on your own legal questions
- Compare with baseline models
- Analyze patterns in reflection tokens
- Use for your DSC261 project!